In [1]:
from tensorflow.keras import Sequential
from tensorflow.keras.datasets import reuters
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
import math

# 파일 불러오기

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [190]:
df = pd.read_csv('drive/My Drive/Colab Notebooks/3rd Project/3rd_Project_Data/TasteScore_TrainDataSet.csv', encoding='utf-16')

In [191]:
df = df.sample(frac=1).reset_index(drop=True)

In [192]:
X = df['customer_review']

In [193]:
y = df['taste']

# 시작

In [196]:
#KoNLPy패키지에는 Hannanum, Kkma, Komoran, Mecab, Okt(Twitter)등의 클래스들이 있습니다
# https://mr-doosun.tistory.com/22
# Okt (Twitter) Class
# 해당 분석기는 오픈 소스 한국어 분석기이고, 과거 트위터 형태소 분석기였습니다.

# KoNLPy 문서에 따른 클래스간의 성능 비교
# https://konlpy.org/ko/v0.4.3/morph/

# morphs  텍스트에서 형태소를 반환한다 / 형태소 : 일정한 의미가 있는 가장 작은 말의 단위
# nouns   텍스트에서 명사를 반환한다
# phrases 텍스트에서 어절을 뽑아낸다
# pos     텍스트에서 품사 정보를 부착하여 반환한다

In [199]:
# okt(Open Korean Text) 
# okt test
# okt.morphs('KoNLPy패키지에는 Hannanum, Kkma, Komoran, Mecab, Okt(Twitter)등의 클래스들이 있습니다.')
# !pip install konlpy
from konlpy.tag import Okt
okt = Okt()

In [200]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 84)

In [201]:
len(X_train),len(y_train), len(X_test), len(y_test)

(43005, 43005, 18431, 18431)

# TF-IDF (+파라미터)

In [202]:
# min-df DF(document-frequency)의 최소 빈도값 <-> max-df
# n-gram 것은 단어의 묶음
# analyzer = 'word'라고 설정시, 학습의 단위를 단어로 (ex - home, go, my ...)
# analyzer = 'char'라고 설정시, 학습의 단위를 글자로 (ex - a, b, c, d ...)
# ngram_range = (1, 2)라고 한다면, 단어의 묶음을 1개부터 2개까지 설정
# go home', 'very good'과 같은 2개 짜리 묶음도 인덱스를 받게 되는 것
# 단어가 묶여야 비로소 의미를 가지는 것들을 위해
# max_feature는 tf-idf vector의 최대 feature를 설정 /  사전에 EDA를 통해서 데이터의 length를 확인

from sklearn.feature_extraction.text import TfidfVectorizer
tfv = TfidfVectorizer(tokenizer=okt.morphs, ngram_range=(1,3), min_df=3, max_df=0.9)
tfv.fit(X_train)
tfv_X_train = tfv.transform(X_train)
tfv_X_train

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


<43005x72116 sparse matrix of type '<class 'numpy.float64'>'
	with 1259616 stored elements in Compressed Sparse Row format>

In [203]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

clf = LogisticRegression(random_state=0)
params = {'C': [1,3,5,7,9]}
grid_cv = GridSearchCV(clf, param_grid = params, cv=4, scoring='accuracy', verbose=1)
grid_cv.fit(tfv_X_train, y_train)

Fitting 4 folds for each of 5 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
 

GridSearchCV(cv=4, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=0, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None, param_grid={'C': [1, 3, 5, 7, 9]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=1)

In [204]:
grid_cv.best_params_

{'C': 5}

In [205]:
grid_cv.best_score_

0.9212882108177368

In [206]:
tfv_X_test=tfv.transform(X_test)
grid_cv.best_estimator_.score(tfv_X_test,y_test)

0.922847376702295

# 테스트

In [207]:
TestSet=pd.read_csv('/content/drive/My Drive/Colab Notebooks/3rd Project/3rd_Project_Data/test_taste_df.csv',encoding='utf-16')

In [208]:
neg_test = TestSet[TestSet['taste']==0]
print("부정: ", len(neg_test))
pos_test = TestSet[TestSet['taste']==1]
pos_test = pos_test.sample(n=500).reset_index(drop=True)
print("긍정: ", len(pos_test))
TestSet = pd.concat([neg_test, pos_test])
print("TestSet: ", len(TestSet))


부정:  405
긍정:  500
TestSet:  905


In [209]:
TestSet = TestSet.sample(frac=1).reset_index(drop=True)
TestSet

,Unnamed: 0,taste,customer_review
0,6192,1.0,맛있어요 번창하세요 최고입니다
1,4832,0.0,맛없어요 돈아까워요. 비추천드려요
2,7920,1.0,맛도 괜찮고 양도 적당합니다 근데 고기 살보다 기름이 더 많아서 아쉬웠습니다
3,1510,1.0,마시써요 ㅋㅋㅋㅋㅋ 잘먹엇어요!!!!!!!
4,7168,0.0,오늘 밥 이상해요.. 딱딱해요
...,...,...,...
900,7208,0.0,명란소스 치킨가라아게 튀김옷이 바삭하지 않고 소스가 없습니다. 트러플향 감자튀김 트...
901,2376,1.0,맛있네요 정말 잘먹었습니다 진짜 맛있어서 소주가 술술~♡♡ ^^잘먹었어요~
902,6536,1.0,간장불고기 ㅈㅁㅌ 나머진soso
903,6012,1.0,역시 맛있는 원할머니 보쌈~~ 서비스로 주신 보쌈 일인분까지 먹으니까 양이 엄청 많...


# test01

In [213]:
test01 = TestSet.iloc[0:10]['customer_review'].tolist()
test01

['맛있어요 번창하세요 최고입니다',
 '맛없어요 돈아까워요. 비추천드려요',
 '맛도 괜찮고 양도 적당합니다 근데 고기 살보다 기름이 더 많아서 아쉬웠습니다',
 '마시써요 ㅋㅋㅋㅋㅋ 잘먹엇어요!!!!!!!',
 '오늘 밥 이상해요.. 딱딱해요',
 '맛있습니다',
 '진짜좋아하는곳이고 배달도 빨리왔는데 달팽이가 나와서 다버렸어요...ㅠ',
 '내 살다 이렇게 개판은 처음이네 하 ㅋㅋㅋㅋㅋ 주문 20분뒤에 전화가옴. 재료가 떨어졌으니 앱으로 다시 주문하라고? 어~ 전화받자마자 바로 영업시간이 끝났네~ 받지도 않아서 한참 뒤에 다시 전화했는데, 다음날 환불될겁니다? 어~ 9일 지나도 환불안돼~ 사람 기다리게하고, 귀찮으니까 멋재로 주문취소하고, 돈은 환불해주기 싫다고?? 직화반상 요기요 직영이라는데 다신 요기요안쓴다 ㅋㅋㅋ 진짜 소비자 호구로 아는 쓰레기 서비스. 이 리뷰 지우기만해봐 아주 ㅋㅋㅋ',
 '진짜 최악임. 아구3등분해서 그냥 처넣어놨음 돈이 아깝다 진짜',
 '찌개는 괜찮은데 고기가좀']

In [214]:
original = TestSet.iloc[0:10]['taste'].tolist()
original_TasteScore = [int(x) for x in original]
print("true : ", original_TasteScore)

my_review = tfv.transform(test01)
pred = grid_cv.best_estimator_.predict(my_review)
print("pred : ", pred)

error = []
for i in range(10):
  x = original_TasteScore[i]-pred[i]
  error.append(int(x))
print("error: ", error)

print("\n")
print("error 값이  1이면 리뷰에 긍정 레이블링인데, 컴퓨터가 부정으로 예측")
print("error 값이 -1이면 리뷰에 부정 레이블링인데, 컴퓨터가 긍정으로 예측")
print("error 값은 실제 리뷰를 보고 판단해보기\n")

num = 0

for i in range(len(error)):
  if error[i] != 0:
    print("error ", error[i], " ", TestSet.iloc[i]['customer_review'])

true :  [1, 0, 1, 1, 0, 1, 0, 0, 0, 0]
pred :  [1. 0. 1. 1. 0. 1. 0. 0. 0. 0.]
error:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


error 값이  1이면 리뷰에 긍정 레이블링인데, 컴퓨터가 부정으로 예측
error 값이 -1이면 리뷰에 부정 레이블링인데, 컴퓨터가 긍정으로 예측
error 값은 실제 리뷰를 보고 판단해보기



# test02

In [216]:
test02 = TestSet.iloc[500:510]['customer_review'].tolist()
test02

['배달도 알림시간에 딱 맞춰 오셨구요. 보쌈 양도, 맛도, 적절하니 딱 맞았구요. 단지 비빔모밀이 떡이었구요 비빔장도 적었어요. ㅠ.ㅠ 가위로 잘게 잘라서 집에 있는 초고추장 더 넣어 비벼먹었답니다. ㅎ 다음엔 물모밀 시켜야겠어요~',
 '잘 먹었습니다~~~',
 '혹시 장조림 비빔밥에 짱아찌 상한거아닌가요.. 쉰내가 심하게 나요...',
 '맛있게 집에서 2차로 잘 먹었습니다',
 '다른 때와 달리 고기가 말랐고 돼지 특유의 냄새가 남',
 '맛은 여전히 맛있습니다!!배달도 빠르고요! 양이 뭔가 아쉬워요 ㅠ',
 '여태껏 이런적이 없었는데 오늘은 고기가 전반적으로 좀 질겼어요.. 담엔 좀더 신경써주셨음 해요ㅜㅜ',
 '배달시킨지 한시간이 넘었는데도 배달도 안오고 전화번호는 기재도 안되어 있고 요기요 서비스는 아예 닫아놨는데 뭘보고 리뷰를 쓰라는건지 모르겠네요',
 '사장님 해물찜에 간을못했나봐요 간이 하나도 안되어 있어요 아구찜은 맛있어요.',
 '맛있고 양이 정말 많아요. 근데 너무 매워요 ㅠ 매운맛을 조절할수 있으면 좋겠어요.']

In [217]:
original = TestSet.iloc[500:510]['taste'].tolist()
original_TasteScore = [int(x) for x in original]
print("true : ", original_TasteScore)

my_review = tfv.transform(test02)
pred = grid_cv.best_estimator_.predict(my_review)
print("pred : ", pred)

error = []
for i in range(10):
  x = original_TasteScore[i]-pred[i]
  error.append(int(x))
print("error: ", error)

print("\n")
print("error 값이  1이면 리뷰에 긍정 레이블링인데, 컴퓨터가 부정으로 예측")
print("error 값이 -1이면 리뷰에 부정 레이블링인데, 컴퓨터가 긍정으로 예측")
print("error 값은 실제 리뷰를 보고 판단해보기\n")

num = 0

for i in range(len(error)):
  if error[i] != 0:
    print("error ", error[i], " ", TestSet.iloc[i]['customer_review'])

true :  [1, 1, 0, 1, 0, 1, 1, 0, 1, 1]
pred :  [1. 1. 0. 1. 0. 1. 1. 0. 0. 1.]
error:  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]


error 값이  1이면 리뷰에 긍정 레이블링인데, 컴퓨터가 부정으로 예측
error 값이 -1이면 리뷰에 부정 레이블링인데, 컴퓨터가 긍정으로 예측
error 값은 실제 리뷰를 보고 판단해보기

error  1   진짜 최악임. 아구3등분해서 그냥 처넣어놨음 돈이 아깝다 진짜


# test03

In [218]:
test03 = TestSet.iloc[200:210]['customer_review'].tolist()
test03

['전에 다른곳시켜먹어봤는데 여기는 고기에 기름도별로없고 맛있네요~~^^',
 '잘 구워진 삼겹살과 버섯, 떡 그리고 마늘...그 위를 감싼 파채까지 정말 너무 마음에 듭니다. 국물과 탄산음료까지 정말 만족스러운 맛과 가격입니다.',
 '싱겁고(양념맛 따로 콩나물맛따로...) 꽃게는 살이 없어요..슬퍼요....새우튀김이 제일 맛있었어요...',
 '양 대비 터무니 없이 비싸네요',
 '뚜껑 열면서 마늘 칸에서 쪽파리 발견...',
 '배달시간 45분지나도안오네요 ㅡㅡ 요기요에서는 리뷰쓰라네요 배달오지도 않았는데 리뷰 쓰라해서 씁니다 배달 예정시간보다 느려요',
 '지난번시켰을때는 괜찬았는데..이번은 먼가바뀐느낌에 맛이 너무없너요..시킨걸 후회합니다..',
 '음...내입맛에는 아닌걸로~~',
 '라라라라라라라라라라',
 '축구보면서 먹는 직구삼은👍']

In [219]:
original = TestSet.iloc[200:210]['taste'].tolist()
original_TasteScore = [int(x) for x in original]
print("true : ", original_TasteScore)

my_review = tfv.transform(test03)
pred = grid_cv.best_estimator_.predict(my_review)
print("pred : ", pred)

error = []
for i in range(10):
  x = original_TasteScore[i]-pred[i]
  error.append(int(x))
print("error: ", error)

print("\n")
print("error 값이  1이면 리뷰에 긍정 레이블링인데, 컴퓨터가 부정으로 예측")
print("error 값이 -1이면 리뷰에 부정 레이블링인데, 컴퓨터가 긍정으로 예측")
print("error 값은 실제 리뷰를 보고 판단해보기\n")

num = 0

for i in range(len(error)):
  if error[i] != 0:
    print("error ", error[i], " ", TestSet.iloc[i]['customer_review'])

true :  [1, 1, 0, 0, 0, 0, 0, 0, 0, 1]
pred :  [0. 1. 0. 1. 0. 0. 0. 0. 1. 1.]
error:  [1, 0, 0, -1, 0, 0, 0, 0, -1, 0]


error 값이  1이면 리뷰에 긍정 레이블링인데, 컴퓨터가 부정으로 예측
error 값이 -1이면 리뷰에 부정 레이블링인데, 컴퓨터가 긍정으로 예측
error 값은 실제 리뷰를 보고 판단해보기

error  1   맛있어요 번창하세요 최고입니다
error  -1   마시써요 ㅋㅋㅋㅋㅋ 잘먹엇어요!!!!!!!
error  -1   진짜 최악임. 아구3등분해서 그냥 처넣어놨음 돈이 아깝다 진짜


# 

# 실제 프로젝트 사용 파일 출력용


In [242]:
restaurnat_list = ['본설렁탕&냉면-신림난곡점', '직화반상by셰플리(관악)', '청년치킨-신림점', '후라이드참잘하는집-독산점' ]

for i in restaurnat_list:

  path = '/content/drive/My Drive/Colab Notebooks/3rd Project/3rd_Project_Data/this/'+ i +'TasteScore_Test.csv'
  df = pd.read_csv(path, encoding='utf-16')
  test = df['customer_review'].tolist()

  review = tfv.transform(test)
  pred = grid_cv.best_estimator_.predict(review)
  pred = pred.tolist()

  df_TrainSet = pd.DataFrame(pred, test)
  df_TrainSet.to_csv('/content/drive/My Drive/Colab Notebooks/3rd Project/3rd_Project_Data/this/' + i + 'TasteScore_pred.csv', encoding='utf-16')

  # 총 댓글 수 / 0 갯수 / 1 갯수 / 0 퍼센트 / 1 퍼센트

  total_review = len(df)
  neg_num = pred.count(0)
  pos_num = pred.count(1)
  neg_per = math.floor(pred.count(0)/len(df)*100)
  pos_per = math.floor(pred.count(1)/len(df)*100)
  print(total_review, neg_num, pos_num, neg_per, pos_per)

2034 213 1821 10 89
2663 478 2185 17 82
1058 47 1011 4 95
1404 115 1289 8 91


# 수동 체크용

In [229]:


path01 = '/content/drive/My Drive/Colab Notebooks/3rd Project/3rd_Project_Data/this/본설렁탕&냉면-신림난곡점TasteScore_Test.csv'
path01 = '/content/drive/My Drive/Colab Notebooks/3rd Project/3rd_Project_Data/this/직화반상by셰플리(관악)TasteScore_Test.csv'
path01 = '/content/drive/My Drive/Colab Notebooks/3rd Project/3rd_Project_Data/this/청년치킨-신림점TasteScore_Test.csv'
path01 = '/content/drive/My Drive/Colab Notebooks/3rd Project/3rd_Project_Data/this/후라이드참잘하는집-독산점TasteScore_Test.csv'

df01 = pd.read_csv(path01, encoding='utf-16')
df02 = pd.read_csv(path02, encoding='utf-16')
df03 = pd.read_csv(path03, encoding='utf-16')
df04 = pd.read_csv(path04, encoding='utf-16')

In [148]:
test01 = df01['customer_review'].tolist()

my_review01 = tfv.transform(test01)
pred01 = grid_cv.best_estimator_.predict(my_review01)
pred01 = pred01.tolist()

In [149]:
test02 = df02['customer_review'].tolist()

my_review02 = tfv.transform(test02)
pred02 = grid_cv.best_estimator_.predict(my_review02)
pred02 = pred02.tolist()

In [150]:
test03 = df03['customer_review'].tolist()

my_review03 = tfv.transform(test03)
pred03 = grid_cv.best_estimator_.predict(my_review03)
pred03 = pred03.tolist()

In [151]:
test04 = df04['customer_review'].tolist()

my_review04 = tfv.transform(test04)
pred04 = grid_cv.best_estimator_.predict(my_review04)
pred04 = pred04.tolist()

In [157]:
df_TrainSet = pd.DataFrame(pred01, test01)
df_TrainSet.to_csv('/content/drive/My Drive/Colab Notebooks/3rd Project/3rd_Project_Data/this/본설렁탕&냉면-신림난곡점TasteScore_pred.csv', encoding='utf-16')

In [158]:
df_TrainSet = pd.DataFrame(pred02, test02)
df_TrainSet.to_csv('/content/drive/My Drive/Colab Notebooks/3rd Project/3rd_Project_Data/this/직화반상by셰플리(관악)TasteScore_pred.csv', encoding='utf-16')

In [159]:
df_TrainSet = pd.DataFrame(pred03, test03)
df_TrainSet.to_csv('/content/drive/My Drive/Colab Notebooks/3rd Project/3rd_Project_Data/this/청년치킨-신림점TasteScore_pred.csv', encoding='utf-16')

In [160]:
df_TrainSet = pd.DataFrame(pred04, test04)
df_TrainSet.to_csv('/content/drive/My Drive/Colab Notebooks/3rd Project/3rd_Project_Data/this/후라이드참잘하는집-독산점TasteScore_pred.csv', encoding='utf-16')

In [175]:
# 총 댓글 수 / 0 갯수 / 1 갯수 / 0 퍼센트 / 1 퍼센트

total_review = len(df01)
neg_num = pred01.count(0)
pos_num = pred01.count(1)
neg_per = math.floor(pred01.count(0)/len(df01)*100)
pos_per = math.floor(pred01.count(1)/len(df01)*100)

In [176]:
print(total_review, neg_num, pos_num, neg_per, pos_per)

2034 227 1807 11 88
